In [1]:
import argparse
from google.cloud import vision
import os
import sys

from detect_text import any_text_triggering
from places import get_places,  get_place_details,  load_zipcodes, get_place_photo
from streetview import get_streetview_image
from store import Store
from vision import extract_image_text
import files
import sql

In [2]:

def load_text(path):
    with open(path, 'r') as f:
        data = f.read().split('\n')
    return data

In [24]:
type([]) == list

True

In [14]:
def _get_place_details(store: Store, verbose = False) -> Store:
    if verbose:
        print('Retrieving advanced details...', end='')
        sys.stdout.flush()
    details = get_place_details(store.place['place_id'])
    if details is None:
        if verbose:
            print('ERROR: Failed to retrieve details for place', store.place['place_id'], end = '')
            sys.stdout.flush()
    # Cheating: checks if details is None first, if false then it won't run next conditional so we won't get erro
    # Or atleast my version of python - Alvin
    store.photos = details['photos'] if details and 'photos' in details else []
    store.reviews = details['reviews'] if details and 'reviews' in details else []
    store.website = details['website'] if  details and'website' in details else None
    if verbose:
        print('Done')
    return store

In [ ]:
def _get_streetview_image(store: Store, images_dir: str, verbose = False) -> Store:
    if verbose:
        print('Getting streetview images... ', end='')
        sys.stdout.flush()
    # Make directory to temporarily store data
    if not os.path.isdir(images_dir): 
        os.mkdir(images_dir)
    image = get_streetview_image(store.place['formatted_address'])
    if image is None:
        name = place['name']
        if verbose:
            print(f'ERROR: Unable to retrieve streetview image for {name}', end='')
            
    else:
        image_path = '{dir}/{place_id}.jpg'.format(
        dir=images_dir,
        place_id=store.place['place_id'])
        with open(image_path, 'wb') as f:
            f.write(image)
    store.image_path = image_path if image else None
    if verbose:
        print('Done')
        sys.stdout.flush()
def _get_review_image(store:Store, images_dir: str, verbose = False) -> Store:
    if verbose:
        print(f'Getting review images...', end = '')
        sys.stdout.flush()
    if not store.photos: # Photos are retrieved from _get_place_details, will not work prior.
        print(f'No list of photos in store.photos, Skipping')
        store.review_images_dir = None
        return store
    review_images_dir = f"{images_dir}/{store.place['place_id']}"
    if not os.path.isdir(review_images_dir):
        os.mkdir(review_images_dir)
    store.review_images_dir = review_images_dir
    for photo in store.photos:
        image_review = get_place_photo(photo['photo_reference'])
        image_path = '{review_images_dir}/{photo_id}.jpg'.format(
                review_images_dir = review_images_dir,
                photo_id=photo['photo_reference'])
        with open(image_path, 'wb') as f:
            f.write(image_review)
    if verbose:
        print(f'Done')
    return store

In [ ]:
# Currently, the review images are not used for text extraction
def _extract_image_text(store: Store, reviews = False, verbose = False) -> Store:
    if verbose:
        print('Extracting text from images... ', end='')
    sys.stdout.flush()
    if store.image_path:
        store.image_text = extract_image_text(store.image_path)
    else:
        if verbose:
            print("No image path...", end='')
        store.image_text = ''
    if reviews:
        pass
    if verbose:
        print(f'Done')
    return store

In [ ]:
def _triggering(store: Store, verbose = False) -> Store:
    # Check descriptions against trigger words
    if verbose:
        print('Checking if store names or extracted text is triggering...', end='')
        sys.stdout.flush()
    # Writing descriptions to file for testing review
    desc_file = open('descriptions.txt', 'w', encoding='utf-8')
    
    name = store.place['name']
    descriptions = store.image_text[1:] #The first index is a the unparsed string, white [1:n] is that same string, split by \n.
    
    strings = [name] + descriptions + sum([x['text'].split(' ') for x in store.reviews], [])
    # Writing to file for review while testing
    desc_file.write(','.join(strings) + '\n')
    store.is_triggering = any_text_triggering(strings, trigger_phrases)
    desc_file.close()
    if verbose:
        print(f'Done')
    return store

In [4]:
# # # #
# Assumes Store has an associated place_id
def _process_store(store: Store, images_dir: str, trigger_phrases: list, reviews_image_text = False, options = 0b11111, verbose = False) -> Store:
    assert(store.place) # Makes sure we don't have a null place
    
    # # # #
    # 
    # https://stackoverflow.com/questions/54342884/check-if-multiple-bits-are-set-or-cleared
    if(options & 0b10000):
        store = _get_place_details(store, verbose)
    if(options & 0b01000): # Doesn't need details
        store = _get_streetview_image(store, images_dir, verbose)
    if((options | ~0b10100) == ~0): #Needs detials to get list og photos
        store = _get_review_image(store, images_dir, verbose)
    if((options | ~0b01010) == ~0 or (options | ~0b00110) == ~0): # Need street_view or review images
        store = _extract_image_text(store, reviews_image_text ,verbose)

    store = _triggering(store, trigger_phrases, verbose)

    return store


SyntaxError: invalid syntax (80885382.py, line 2)

In [ ]:
if __name__ == '__main__':
    # Functional Arguments
    parser = argparse.ArgumentParser(description='Pipeline produced by Fall 2023 UofL Capstone Team for Tobacco Permits')
    parser.add_argument('-i', '--images_dir', action='store', default='streetview_images', dest = 'images_dir',
                        help='Directory to save Google Streetview Images to')
    parser.add_argument('-z', '--zip_codes', action='store', default = 'louisville_metro_zipcodes.txt', dest='zip_codes',
                        help='.txt file to read in selected zipcodes to run algorithm for.')
    parser.add_argument('-s','--store_types', action = 'store', default= 'store_types.txt', dest = 'store_types',
                        help='Types of stores to be identified using google places.')
    parser.add_argument('-t','--trigger_phrases', action = 'store', default= 'trigger_phrases.txt', dest = 'trigger_phrases',
                        help='Words that are algorithm will look for to \"Flag\" a store.')
    # Testing Arguments
    # TO-DO: Change "real" to be a limit
    parser.add_argument('-r', '--real', action = 'store_true', dest = 'real',
                        help = 'Run algorithm on entire dataset over just 1 value')
    parser.add_argument('-v', '--verbose', action = 'store_true', dest = 'verbose',
                        help ='Be verbose (print statements)')
    parser.add_argument('-l', '--logging', action = 'store_true',  dest = 'logging',
                        help ='Log progress data')
    parser.add_argument('-ld', '--log_dir', action = 'store', default = 'logs' , dest='log_dir',
                        help = 'Where to log')
    parser.add_argument('-c', '--cache', action = 'store_true', dest='cache',
                        help = 'Save data into database and filese')
    parser.add_argument('-sd', '--store_dir', action = 'store', default = 'store_data', dest = 'store_dir',
                        help='If caching, where to save store data')
    # PULLING ARGS
    #
    #
    #
    args = parser.parse_args()
    images_dir = args.images_dir
    zipcodes = load_text(args.zip_codes)
    store_types = load_text(args.store_types)
    trigger_phrases = load_text(args.trigger_phrases)

    # Get places for all store types in all zipcodes
    if args.verbose:
        print('Getting places... ', end='')
    print('Full version:', args.real)
    sys.stdout.flush()

# Testing

In [1]:
import main

In [22]:
any([True, None])

True

In [1]:
#https://stackoverflow.com/questions/54342884/check-if-multiple-bits-are-set-or-cleared
options = 0b11101
(options | ~0b10100 )

-1

In [2]:
stores = main.get_stores(['40214'],['smoke'])
stores[0] = main._process_store(stores[0], 'streetview_images', main.load_text('trigger_phrases.txt'),
                                False, 0b11101, False)

In [3]:
print(stores[0])

Store: Third Smoking Shop
,
              len(photos): 10 
,
              image_path: streetview_images/ChIJI9HPiX8NaYgR46NU1o0A8Q0.jpg
,
              image_text: None
,
              is_triggering: True
,
              len(reviews): 5
,
              website: None


In [4]:
 main.get_stores(['louisville'],['smoke'])

In [8]:
import os
for root, dirs, files in os.walk('test'):
    for name in files:
        print(os.path.join(root, name))


test\images\cigarette_wall_display.jpg
test\images\fat_rabbit.jpg
test\images\gas_station_outside.jpg
test\images\vape_shop_counter_01.jpg
test\images\wave_ad_outside_of_corner_store.jpg


In [19]:
i = 10
while True:
    print(i)
    if i:
        i -= 1
        if i == 0: break 
        else:  pass
        

10
9
8
7
6
5
4
3
2
1
